In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import wandb
from scripts.models import *
from scripts.utils import custom_metric, Wandb_plot, csv_logger, animate_function, early_stopping
import pandas as pd
import matplotlib.animation as animation
import ast
import networkx as nx
from matplotlib.animation import FuncAnimation, PillowWriter

2024-08-25 17:19:23.557405: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-25 17:19:23.673482: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-25 17:19:23.706551: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-25 17:19:23.951483: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-25 17:19:25.144539: W tensorflow/compiler/tf2

In [2]:
m = Model(1, [1,512,512,512,1], [None, tf.keras.activations.tanh,tf.keras.activations.tanh,tf.keras.activations.tanh,None])
m

[layer : layer_0, neurons: 1, layer : layer_1, neurons: 512, layer : layer_2, neurons: 512, layer : layer_3, neurons: 512, layer : layer_4, neurons: 1]

In [3]:
wandb.login()
run = wandb.init(
    # Set the project where this run will be logged
    project="Neural Network from scratch", 
    # Track hyperparameters and run metadata
    config={
        "learning_rate": 0.01,
        "epochs": 1500,
        "batch_size": 100,
        "activation": "Tanh",
        "architecture" : "1,512,512,512,1",
        "Optimizer": "Adam",
        "Name" : "Tanh Model"
    },
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: a01700257. Use `wandb login --relogin` to force relogin


In [4]:
x_train = np.arange(-10, 10, 0.01)
y_train = np.sin(x_train)
y_mean = np.mean(y_train)

x_train = tf.convert_to_tensor(x_train, dtype=tf.float32)
x_train = tf.reshape(x_train, (-1,1))
y_train = tf.convert_to_tensor(y_train, dtype=tf.float32)
y_train = tf.reshape(y_train, (-1,1))

I0000 00:00:1724606469.166281    9276 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1724606469.203646    9276 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1724606469.203822    9276 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1724606469.207143    9276 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

In [5]:
# Define learning rate schedule and optimizer
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.01,
    decay_steps=1000,
    decay_rate=0.9)

loss_metric = custom_metric(name = "loss")
accuracy_metric = custom_metric(name = "accuracy")

metrics = {'loss': loss_metric, 'accuracy': accuracy_metric}

optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)
m.compile(optimizer, metrics, y_mean)
w_callback = Wandb_plot(epoch_plot= 10)
logs = csv_logger(name = "tanh.csv")
early_stop = early_stopping()

In [6]:
m.fit(x_train, y_train, epochs=1500, batch_size=100, callbacks=[logs, w_callback], verbose=0)
wandb.finish()

I0000 00:00:1724606736.392588    9403 service.cc:146] XLA service 0x7f6564005960 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1724606736.392608    9403 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce GTX 1050 Ti, Compute Capability 6.1
2024-08-25 11:25:44.040472: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-08-25 11:26:14.156528: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907
2024-08-25 11:29:42.568816: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'input_add_reduce_fusion_7', 1252 bytes spill stores, 1228 bytes spill loads

I0000 00:00:1724606983.881680    9403 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the proc

Early stopping at epoch 19 with loss 45.932579040527344


Accuracy,██████▆▅▅▅▅▅▅▅▅▆▅▄▆▅▄▃▄▅▆▅▆▅▅▆▃▁▅▆▅▅▅▆▆▅
Loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Accuracy,-22.0315
Loss,47.42049


In [2]:
animate_function(logs= 'logs/tanh.csv', name = "tanh.gif")

MovieWriter imagemagick unavailable; using Pillow instead.
